# Connecting to the database

The customer of this research is a large telecommunications company that provides services throughout the CIS. The company is faced with the task of determining the current level of consumer loyalty, or NPS (from the English Net Promoter Score), among customers from Russia.
To determine the level of loyalty, customers were asked a classic question: “On a scale of 1 to 10, rate the likelihood that you would recommend the company to friends and acquaintances.”
The company conducted a survey and asked me to prepare a dashboard with its results. They did not deploy a large database for such a task and uploaded the data to SQLite.

In [1]:
# import the libraries necessary for further work

import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# paths to potential database locations
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

# check if the local database exists; if not, check the platform database
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local  # use local database if available
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform  # fallback to platform database if local is absent
else:
    # raise an exception if no database is found at either location
    raise Exception('Файл с базой данных SQLite не найден!')

# initialize database connection if a valid database path was determined
if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)  # create SQLAlchemy engine without logging


# Loading data

In [3]:
query = \
"""
-- define a common table expression (CTE) named ExtendedUser
WITH ExtendedUser AS (
  SELECT
    *,
    -- determine if the user is new based on the 'lt_day' field
    CASE
      WHEN lt_day >= 365 THEN 'no'
      ELSE 'yes'
    END as is_new,
    -- classify users into NPS groups based on their 'nps_score'
    CASE
      WHEN nps_score <= 6 THEN 'detractors'
      WHEN nps_score BETWEEN 7 AND 8 THEN 'passives'
      WHEN nps_score >= 9 THEN 'promoters'
    END as nps_group,
    -- convert 'gender_segment' numerical codes to readable text
    CASE
      WHEN gender_segment == 1 THEN 'female'
      ELSE 'male'
    END as gender_segment_w
  FROM
    user
)

-- main SELECT query to retrieve user data along with related dimensional data
SELECT
  u.user_id,
  u.lt_day,
  u.is_new,
  u.age,
  u.gender_segment_w,
  u.os_name,
  u.cpe_type_name,
  l.country,
  l.city,
  ags.title as age_segment,
  ts.title as traffic_segment,
  ls.title as lifetime_segment,
  u.nps_score,
  u.nps_group
FROM
  ExtendedUser as u  -- use the CTE defined earlier
  JOIN location as l ON u.location_id = l.location_id 
  JOIN age_segment as ags ON u.age_gr_id = ags.age_gr_id  
  JOIN traffic_segment as ts ON u.tr_gr_id = ts.tr_gr_id  
  JOIN lifetime_segment as ls ON u.lt_gr_id = ls.lt_gr_id;  

"""

In [4]:
# check 
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment_w,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,no,45.0,female,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,no,53.0,male,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,no,57.0,male,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters
3,A004TB,4190,no,44.0,female,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,promoters
4,A004XT,1163,no,24.0,male,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,promoters
5,A005O0,5501,no,42.0,female,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,detractors
6,A0061R,1236,no,45.0,male,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,promoters
7,A009KS,313,yes,35.0,male,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,promoters
8,A00AES,3238,no,36.0,female,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,promoters
9,A00F70,4479,no,54.0,female,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,promoters


In [5]:
# exporting as csv to continue work in data-viz
df.to_csv('telecomm_csi_tableau.csv', index=False)

# Tableau

All other work related to analytics and explication of conclusions and insights was done in Tableau.

The project is presented in the form of a story. Each slide presents one specific topic and is accompanied by conclusions and observations. There are three slides in total: about the history of the speeches, about the topics of the speeches and about the authors of the speeches. Each slide is a separate dashboard with active action elements.

Data-viz is [here](https://public.tableau.com/app/profile/artem.aglyamov/viz/projectNo_2TELECOMNPS/Story1 "yep, that's right, tap on me")